# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

driver = webdriver.Chrome()

/Users/shaneburke/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for **cosmetologist violations** for people with the last name **Nguyen**.

In [2]:
driver.get("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

cosmetologist_field = driver.find_element_by_xpath("/html/body/div/div/div[2]/div[1]/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[3]/td/select/option[10]")
cosmetologist_field.click()

last_name = driver.find_element_by_id("pht_lnm")
last_name.send_keys("Nguyen")

submit_button = driver.find_element_by_xpath("/html/body/div/div/div[2]/div[1]/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]")
submit_button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [3]:
table = driver.find_element_by_tag_name("tbody")
rows = table.find_elements_by_tag_name("tr")
for row in rows[:10]:
    print(row.text.strip())

Name and Location Order Basis for Order
NGUYEN, MIMI PHAM
City: KATY
County: HARRIS
Zip Code: 77449


License #: 784210

Complaint # COS20190010072 Date: 11/12/2020

Respondent is assessed an administrative penalty in the amount of $1,125. Respondent failed properly clean and sanitize the metal implements used at the Salon; Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution.
NGUYEN, HA
City: ARLINGTON
County: TARRANT
Zip Code: 76017


License #: 764888

Complaint # COS20190016762 Date: 11/12/2020

Respondent is assessed an administrative penalty in the amount of $2,250. Respondent failed to clean and sanitize four (4) whirlpool foot spas as required at the end of each day, constituting two (2) violations; Respondent failed to keep a record of the date and time of four (4) foot spas daily or bi-weekly cleaning and if the foot spas were not used, constituting two (2) violations.
NGUYEN, THAO HONG
City: SAN ANTONIO
County: BEXAR
Zip

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   # Instead of stopping on an error, it'll jump down here instead
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement. Most people use `pass`, but it's also nice to print out debug statements so you know when/where it's running into errors.

**Why doesn't the first one have a name?**

In [4]:
for row in rows[:10]:
    try:
        print(row.find_elements_by_class_name("results_text")[0].text.strip())
    except:
        print("it didn't work")
        
        
#First one is the heading

it didn't work
NGUYEN, MIMI PHAM
NGUYEN, HA
NGUYEN, THAO HONG
NGUYEN, CINDY
NGUYEN, CHAU KHANH LINH
NGUYEN, TRANG T
NGUYEN, DUNG MINH
NGUYEN, YEN NHI THI
NGUYEN, JOHNNY DAT


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...*

In [5]:
#got rid of heading and added tenth row

for row in rows[1:11]:
    try:
        print(row.find_elements_by_tag_name("td")[2].text.strip())
        print("~*~*~")
    except:
        print("oops")

Respondent failed properly clean and sanitize the metal implements used at the Salon; Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution.
~*~*~
Respondent failed to clean and sanitize four (4) whirlpool foot spas as required at the end of each day, constituting two (2) violations; Respondent failed to keep a record of the date and time of four (4) foot spas daily or bi-weekly cleaning and if the foot spas were not used, constituting two (2) violations.
~*~*~
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
~*~*~
Respondent failed to clean and disinfect all wax pots; Respondent failed to properly clean multi-use items prior to each service.
~*~*~
Respondent engaged in fraud or deceit in obtaining a certificate, license, or permit.
~*~*~
Respondent failed properly clean and sanitize the metal implements used at the Salon; Respondent failed to wipe clean and disinfect electrical e

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [6]:
for row in rows[1:11]:
    try:
        print(row.find_elements_by_class_name("results_text")[5].text.strip())
    except:
        print("it didn't work")

COS20190010072
COS20190016762
COS20200010387
COS20200010502
COS20190008104
COS20200010511
COS20200004202
COS20190004199
COS20200000101
COS20200011664


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [7]:
cases = []

for row in rows[1:11]:
    name = row.find_elements_by_class_name("results_text")[0].text.strip()
    description = row.find_elements_by_tag_name("td")[2].text.strip()
    violation_number = row.find_elements_by_class_name("results_text")[5].text.strip()
    license_number = row.find_elements_by_class_name("results_text")[4].text.strip()
    zip_code = row.find_elements_by_class_name("results_text")[3].text.strip()
    county = row.find_elements_by_class_name("results_text")[2].text.strip()
    city = row.find_elements_by_class_name("results_text")[1].text.strip()
    
    cases.append({'name' : name,
                  'description' : description, 
                  'violation number' : violation_number,
                  'license number' : license_number,
                  'zip code' : zip_code,
                  'county' : county,
                  'city' : city})

print(cases)
                  

[{'name': 'NGUYEN, MIMI PHAM', 'description': 'Respondent failed properly clean and sanitize the metal implements used at the Salon; Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution.', 'violation number': 'COS20190010072', 'license number': '784210', 'zip code': '77449', 'county': 'HARRIS', 'city': 'KATY'}, {'name': 'NGUYEN, HA', 'description': 'Respondent failed to clean and sanitize four (4) whirlpool foot spas as required at the end of each day, constituting two (2) violations; Respondent failed to keep a record of the date and time of four (4) foot spas daily or bi-weekly cleaning and if the foot spas were not used, constituting two (2) violations.', 'violation number': 'COS20190016762', 'license number': '764888', 'zip code': '76017', 'county': 'TARRANT', 'city': 'ARLINGTON'}, {'name': 'NGUYEN, THAO HONG', 'description': 'Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.'

### Save that to a CSV

- Tip: Use `pd.DataFrame` to create a dataframe, and then save it to a CSV.

In [8]:
df = pd.DataFrame(cases)

df

,name,description,violation number,license number,zip code,county,city
0,"NGUYEN, MIMI PHAM",Respondent failed properly clean and sanitize ...,COS20190010072,784210,77449,HARRIS,KATY
1,"NGUYEN, HA",Respondent failed to clean and sanitize four (...,COS20190016762,764888,76017,TARRANT,ARLINGTON
2,"NGUYEN, THAO HONG","Respondent failed to clean, disinfect, and ste...",COS20200010387,"799926, 1753491",78238,BEXAR,SAN ANTONIO
3,"NGUYEN, CINDY",Respondent failed to clean and disinfect all w...,COS20200010502,"806232, 1260359, 1280071",78414,NUECES,CORPUS CHRISTI
4,"NGUYEN, CHAU KHANH LINH",Respondent engaged in fraud or deceit in obtai...,COS20190008104,1764073,36116,OUT OF STATE,MONTGOMERY
5,"NGUYEN, TRANG T",Respondent failed properly clean and sanitize ...,COS20200010511,748483,78155,GUADALUPE,SEGUIN
6,"NGUYEN, DUNG MINH",Respondent failed properly clean and sanitize ...,COS20200004202,785878,77066,HARRIS,HOUSTON
7,"NGUYEN, YEN NHI THI",Respondent failed to clean and disinfect all w...,COS20190004199,763645,78717,TRAVIS,AUSTIN
8,"NGUYEN, JOHNNY DAT",Respondent failed to clean and sanitize whirlp...,COS20200000101,797651,78574,HIDALGO,MISSION
9,"NGUYEN, KELLY PHUONG N",Respondent failed properly clean and sanitize ...,COS20200011664,751553,78413,NUECES,CORPUS CHRISTI


In [9]:
df.to_csv(r'/Users/shaneburke/Desktop/Data Foundations/10-homework/texas_cosmetology_violations.csv', index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [10]:
csv_check = pd.read_csv("texas_cosmetology_violations.csv")
csv_check.head(10)

,name,description,violation number,license number,zip code,county,city
0,"NGUYEN, MIMI PHAM",Respondent failed properly clean and sanitize ...,COS20190010072,784210,77449,HARRIS,KATY
1,"NGUYEN, HA",Respondent failed to clean and sanitize four (...,COS20190016762,764888,76017,TARRANT,ARLINGTON
2,"NGUYEN, THAO HONG","Respondent failed to clean, disinfect, and ste...",COS20200010387,"799926, 1753491",78238,BEXAR,SAN ANTONIO
3,"NGUYEN, CINDY",Respondent failed to clean and disinfect all w...,COS20200010502,"806232, 1260359, 1280071",78414,NUECES,CORPUS CHRISTI
4,"NGUYEN, CHAU KHANH LINH",Respondent engaged in fraud or deceit in obtai...,COS20190008104,1764073,36116,OUT OF STATE,MONTGOMERY
5,"NGUYEN, TRANG T",Respondent failed properly clean and sanitize ...,COS20200010511,748483,78155,GUADALUPE,SEGUIN
6,"NGUYEN, DUNG MINH",Respondent failed properly clean and sanitize ...,COS20200004202,785878,77066,HARRIS,HOUSTON
7,"NGUYEN, YEN NHI THI",Respondent failed to clean and disinfect all w...,COS20190004199,763645,78717,TRAVIS,AUSTIN
8,"NGUYEN, JOHNNY DAT",Respondent failed to clean and sanitize whirlp...,COS20200000101,797651,78574,HIDALGO,MISSION
9,"NGUYEN, KELLY PHUONG N",Respondent failed properly clean and sanitize ...,COS20200011664,751553,78413,NUECES,CORPUS CHRISTI


## Let's do this an easier way

Use Selenium and `pd.read_html` to get the table as a dataframe.

In [11]:
new = pd.read_html(driver.page_source)[0]
#Saw page source here: https://stackoverflow.com/questions/45394374/trying-to-scrape-table-using-pandas-from-seleniums-result

In [12]:
new

,Name and Location,Order,Basis for Order
0,"NGUYEN, MIMI PHAM City: KATY County: HARRIS Zi...",Date: 11/12/2020Respondent is assessed an admi...,Respondent failed properly clean and sanitize ...
1,"NGUYEN, HA City: ARLINGTON County: TARRANT Zip...",Date: 11/12/2020Respondent is assessed an admi...,Respondent failed to clean and sanitize four (...
2,"NGUYEN, THAO HONG City: SAN ANTONIO County: BE...",Date: 11/12/2020Respondent is assessed an admi...,"Respondent failed to clean, disinfect, and ste..."
3,"NGUYEN, CINDY City: CORPUS CHRISTI County: NUE...",Date: 10/29/2020Respondent is assessed an admi...,Respondent failed to clean and disinfect all w...
4,"NGUYEN, CHAU KHANH LINH City: MONTGOMERY Count...",Date: 10/26/2020The Respondent's Cosmetology M...,Respondent engaged in fraud or deceit in obtai...
...,...,...,...
152,"NGUYEN, SHARON City: BASTROP County: BASTROP Z...",Date: 9/17/2018Respondent is assessed an admin...,Respondent leased space in a salon to an indiv...
153,"NGUYEN, BINH THANH City: LAREDO County: WEBB Z...",Date: 9/12/2018Respondent is assessed an admin...,Respondent leased space in a salon to an indiv...
154,"NGUYEN, SAMANTHA TRAN City: MCKINNEY County: C...",Date: 9/4/2018Respondent is assessed an admini...,"Respondent failed to clean, disinfect, and ste..."
155,"NGUYEN, THU LE City: SAN ANTONIO County: BEXAR...",Date: 8/7/2018The Respondent's Cosmetology Man...,Respondent failed to comply with an order prev...
